# TP : Analyse de sentiments dans les critiques de films

## Objectifs

1. Représenter des données textuelles de manière simple.
2. Utiliser un modèle d'apprentissage statistique basique pour une tâche d'analyse de sentiments.
3. Tenter d'améliorer ces représentations avec des outils venus du traitement automatique du langage
5. Comparer les résultats avec une méthode coûteuse état-de-l'art: *fine-tuner* un modèle *Bert*.

## Dépendances nécessaires

Pour les objectifs 2. et 3., on aura besoin des packages suivants:
- The Machine Learning API Scikit-learn : http://scikit-learn.org/stable/install.html
- The Natural Language Toolkit : http://www.nltk.org/install.html

Les deux sont disponibles avec Anaconda: https://anaconda.org/anaconda/nltk et https://anaconda.org/anaconda/scikit-learn

Pour l'objectif 4., il faudra installer le package ```transformers```: https://huggingface.co/transformers/index.html

In [41]:
import os.path as op
import re 
import numpy as np
import matplotlib.pyplot as plt
import gdown

In [1]:
!pip install transformers

In [3]:
!pip install gdown

In [42]:
gdown.download("http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", output="aclImdb_v1.tar.gz", quiet=False)
#!tar xzf /content/aclImdb_v1.tar.gz

Downloading...
From: http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
To: /home/cephas/Documents/TP_PYTHON/Celia/aclImdb_v1.tar.gz
100%|██████████| 84.1M/84.1M [00:21<00:00, 3.87MB/s]


'aclImdb_v1.tar.gz'

## Charger les données

On récupère les données textuelles dans la variable *train_texts*;

On récupère les labels dans la variable $train_labels$: $0$ indique que la critique correspondante est négative tandis que $1$ qu'elle est positive.

In [46]:
from glob import glob
# We get the files from the path: ./data/imdb1/neg for negative reviews, and ./data/imdb1/pos for positive reviews
train_filenames_neg = sorted(glob(op.join('.', 'aclImdb', 'train', 'neg', '*.txt')))
train_filenames_pos = sorted(glob(op.join('.', 'aclImdb', 'train', 'pos', '*.txt')))

"""
test_filenames_neg = sorted(glob(op.join('.', 'aclImdb', 'test', 'neg', '*.txt')))
test_filenames_pos = sorted(glob(op.join('.', 'aclImdb', 'test', 'pos', '*.txt')))
"""

# Each files contains a review that consists in one line of text: we put this string in two lists, that we concatenate
train_texts_neg = [open(f, encoding="utf8").read() for f in train_filenames_neg]
train_texts_pos = [open(f, encoding="utf8").read() for f in train_filenames_pos]
train_texts = train_texts_neg + train_texts_pos

"""
test_texts_neg = [open(f, encoding="utf8").read() for f in test_filenames_neg]
test_texts_pos = [open(f, encoding="utf8").read() for f in test_filenames_pos]
test_texts = test_texts_neg + test_texts_pos
"""

# The first half of the elements of the list are string of negative reviews, and the second half positive ones
# We create the labels, as an array of [1,len(texts)], filled with 1, and change the first half to 0
train_labels = np.ones(len(train_texts), dtype=np.int)
train_labels[:len(train_texts_neg)] = 0.

"""
test_labels = np.ones(len(test_texts), dtype=np.int)
test_labels[:len(test_texts_neg)] = 0.
"""

'\ntest_labels = np.ones(len(test_texts), dtype=np.int)\ntest_labels[:len(test_texts_neg)] = 0.\n'

Exemple de document: 

In [45]:
open("./aclImdb/train/neg/0_3.txt", encoding="utf8").read()

"Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly."

**Dans tout ce TP, l'impact de vos choix sur les résultats dépendra grandement de la quantité de données utilisée:**
essayez de faire varier le paramètre ```k``` dans l'encart suivant !

In [47]:
# This number of documents may be high for most computers: we can select a fraction of them (here, one in k)
# Use an even number to keep the same number of positive and negative reviews
k = 1
train_texts_reduced = train_texts[0::k]
train_labels_reduced = train_labels[0::k]

print('Nombre de documents:', len(train_texts_reduced))

Nombre de documents: 25000


On peut utiliser une fonction utilitaire de sklearn, ```train_test_split```, pour la séparation des données en ensembles d'entraînement et de validation:

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
train_texts_splt, val_texts, train_labels_splt, val_labels = train_test_split(train_texts_reduced, train_labels_reduced, test_size=.2)

In [50]:
train_texts_splt

["I just did not enjoy this film. But then I loved Babe, a Pig in the City and have been spoiled by talking animal films that are exceptionally well done in every way. The animals were not likeable. They were all irritating especially Chris Rock's guinea pig, but then what could I expect, it's Chris Rock. I believe I smiled once or twice at a couple cute lines, but that's it.",
 'Also known as "Stairway to Heaven" in the US. During WWII British Peter Carter\'s (David Niven) plane is shot down in combat but he survives. He meets and falls in love with lovely June (Kim Hunter). But it seems a mistake was made in Heaven--he should have died! A French spirit comes to get him but he refuses. He is soon to plead his case in front of a Heavenly Tribunal that he should be allowed to live.<br /><br />Sounds ridiculous but this is actually an incredible film. The script is good with the actors playing the roles completely straight-faced and it\'s beautifully directed--the scenes on Earth are in 

## Représentation adaptée des documents

Notre modèle, comme la plupart des modèles appliqués aux données textuelles, utilise les comptes d'occurences de mots dans un document. Ainsi, une manière très pratique de représenter un document est d'utiliser un vecteur "Bag-of-Words" (BoW), contenant les comptes de chaque mot (indifférement de leur ordre d'apparition) dans le document. 

Si on considère l'ensemble de tous les mots apparaissant dans nos $T$ documents d'apprentissage, que l'on note $V$ (Vocabulaire), on peut créer **un index**, qui est une bijection associant à chaque mot $w$ un entier, qui sera sa position dans $V$. 

Ainsi, pour un document extrait d'un ensemble de documents contenant $|V|$ mots différents, une représentation BoW sera un vecteur de taille $|V|$, dont la valeur à l'indice d'un mot $w$ sera son nombre d'occurences dans le document. 

On peut utiliser la classe **CountVectorizer** de scikit-learn pour obtenir de telles représentations:

In [51]:
from sklearn.feature_extraction.text import CountVectorizer

Exemple d'utilisation:

In [52]:
corpus = ['I walked down down the boulevard',
          'I walked down the avenue',
          'I ran down the boulevard',
          'I walk down the city',
          'I walk down the the avenue']
vectorizer = CountVectorizer()

Bow = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())
Bow.toarray()

['avenue', 'boulevard', 'city', 'down', 'ran', 'the', 'walk', 'walked']


array([[0, 1, 0, 2, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 1, 1, 1, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0],
       [1, 0, 0, 1, 0, 2, 1, 0]])

Attention: vérifiez la mémoire que les représentations vont occuper (étant donné la façon dont elles sont construites). Quel argument de ```CountVectorizer``` permet d'éviter le problème ? 

In [ ]:
# Create and fit the vectorizer to the training data


In [ ]:
# Look at the shape of the obtained array


In [ ]:
# Transform the validation data


On va utiliser le ```MultinomialNB``` de scikit-learn, une implémentation du modèle Bayésien naïf. Ici, l'hypothèse naïve est que les différentes variables (mots) d'une critique sont indépendantes entre elles.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Create the model, train it and do the prediction 


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

In [ ]:
# Show the results in a readable format


On peut jeter un oeil aux *features* construites par le ```vectorizer```. Comment pourrait-on les améliorer ? 

In [ ]:
print(vectorizer.get_feature_names()[:100])

### Améliorer les représentations BoW

Les arguments du ```vectorizer``` vont nous permettre d'agir facilement sur la façon dont les données sont représentées. On peut donc chercher à améliorer nos représentations *Bag-of-words*:

#### Ne pas prendre en compte les mots trop fréquents:

On peut utiliser l'option ```max_df=1.0``` pour modifier la quantité de mots pris en compte. 

#### Essayer différentes granularités:

Plutôt que de simplement compter les mots, on peut compter les séquences de mots - de taille limitée, bien sur. 
On appelle une séquence de $n$ mots un $n$-gram: essayons d'utiliser les 2 et 3-grams (bi- et trigrams).
On peut aussi tenter d'utiliser les séquences de caractères à la place de séquences de mots.

On s'intéressera aux options ```analyzer='word'``` et ```ngram_range=(1, 2)``` que l'on changera pour modifier la granularité. 

**Encore une fois: avoir recours à ces features aura probablement plus d'impact si la quantité de données d'entraînement à notre disposition est limitée !**

Pour accélérer les expériences, utilisez l'outil ```Pipeline``` de scikit-learn. 

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# Create a pipeline for each configuration


#### Tf-idf:

Il s'agit du produit de la fréquence du terme (TF) et de sa fréquence inverse dans les documents (IDF).
Cette méthode est habituellement utilisée pour extraire l'importance d'un terme $i$ dans un document $j$ relativement au reste du corpus, à partir d'une matrice d'occurences $mots \times documents$. Ainsi, pour une matrice $\mathbf{T}$ de $|V|$ termes et $D$ documents:
$$\text{TF}(T, w, d) = \frac{T_{w,d}}{\sum_{w'=1}^{|V|} T_{w',d}} $$

$$\text{IDF}(T, w) = \log\left(\frac{D}{|\{d : T_{w,d} > 0\}|}\right)$$

$$\text{TF-IDF}(T, w, d) = \text{TF}(X, w, d) \cdot \text{IDF}(T, w)$$

On peut l'adapter à notre cas en considérant que le contexte du deuxième mot est le document. Cependant, TF-IDF est généralement plus adaptée aux matrices peu denses, puisque cette mesure pénalisera les termes qui apparaissent dans une grande partie des documents. 

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# You can use Pipeline here too


### Outils de prétraitement: NLTK

On va maintenant chercher à pré-traiter nos données de manière à simplifier la tâche du modèle. **Notez toujours que celà ne sera probablement utile que si l'on a trop peu de données à disposition !**

#### Stemming: 

Permet de revenir à la racine d'un mot: on peut ainsi grouper différents mots autour de la même racine, ce qui facilite la généralisation. On peut utiliser l'outil ```SnowballStemmer```.

In [ ]:
from nltk import SnowballStemmer
stemmer = SnowballStemmer("english")

Exemple d'utilisation:

In [ ]:
words = ['singers', 'cat', 'generalization', 'philosophy', 'psychology', 'philosopher']
for word in words:
    print('word : %s ; stemmed : %s' %(word, stemmer.stem(word)))#.decode('utf-8'))))

Transformation des données:

In [ ]:
def stem(X): 
# To complete !

In [ ]:
# Test it on the dataset


#### Partie du discours

Pour généraliser, on peut aussi utiliser les parties du discours (Part of Speech, POS) des mots, ce qui nous permettra  de filtrer l'information qui n'est potentiellement pas utile au modèle. On va récupérer les POS des mots à l'aide des fonctions ```pos_tag, word_tokenize```.

In [ ]:
import nltk
from nltk import pos_tag, word_tokenize

Exemple d'utilisation:

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

pos_tag(word_tokenize(('I am Sam')))

Transformation des données:

Ne garder que les noms, adverbes, verbes et adjectifs (```['NN', 'VB', 'ADJ', 'RB']```) pour notre modèle. 

In [ ]:
def pos_tag_filter(X, good_tags=['NN', 'VB', 'ADJ', 'RB']):
# To complete !

In [ ]:
# Test it on the dataset


## Fine-tuning d'un modèle Bert

Suivant le modèle du TP précédent, fine-tunez le modèle Bert le plus léger disponible sur les données d'IMDB et comparez avec les résultats obtenus précédemment. **Encore une fois, si possible, réalisez cette étude pour des quantités de données différentes.**

In [ ]:
import transformers